<a href="https://colab.research.google.com/github/ArjunNarayan2066/CS484_project/blob/fixed_training_loop/cs484_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies
! nvcc --version

! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install tensorboardX==1.4
! pip install opencv-python==3.3.1.11

# Clone repo
! git clone https://github.com/nianticlabs/monodepth2.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py", line 438, in run
    self._warn_about_co

In [3]:
# ! python monodepth2/test_simple.py --image_path monodepth2/assets/test_image.jpg --model_name mono+stereo_640x192 --model_path /root/tmp/S_640x192/models/weights_19
# /root/tmp/S_640x192/models/weights_9/encoder.pth

In [4]:
# ! rm -rf *.zip
# ! rm -rf kitti_data


In [5]:
"""
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0002/2011_09_26_drive_0002_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0005/2011_09_26_drive_0005_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0009/2011_09_26_drive_0009_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0011/2011_09_26_drive_0011_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0013/2011_09_26_drive_0013_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0014/2011_09_26_drive_0014_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0015/2011_09_26_drive_0015_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0017/2011_09_26_drive_0017_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0018/2011_09_26_drive_0018_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0019/2011_09_26_drive_0019_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0020/2011_09_26_drive_0020_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0022/2011_09_26_drive_0022_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0023/2011_09_26_drive_0023_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0027/2011_09_26_drive_0027_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0028/2011_09_26_drive_0028_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0029/2011_09_26_drive_0029_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0032/2011_09_26_drive_0032_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0035/2011_09_26_drive_0035_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0036/2011_09_26_drive_0036_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0039/2011_09_26_drive_0039_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0046/2011_09_26_drive_0046_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0048/2011_09_26_drive_0048_sync.zip
! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0051/2011_09_26_drive_0051_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0052/2011_09_26_drive_0052_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0056/2011_09_26_drive_0056_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0057/2011_09_26_drive_0057_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0059/2011_09_26_drive_0059_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0060/2011_09_26_drive_0060_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0061/2011_09_26_drive_0061_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0064/2011_09_26_drive_0064_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0070/2011_09_26_drive_0070_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0079/2011_09_26_drive_0079_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0084/2011_09_26_drive_0084_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0086/2011_09_26_drive_0086_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0087/2011_09_26_drive_0087_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0091/2011_09_26_drive_0091_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0093/2011_09_26_drive_0093_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0095/2011_09_26_drive_0095_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0096/2011_09_26_drive_0096_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0101/2011_09_26_drive_0101_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0104/2011_09_26_drive_0104_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0106/2011_09_26_drive_0106_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0113/2011_09_26_drive_0113_sync.zip
# # ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0117/2011_09_26_drive_0117_sync.zip

! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip
"""

'\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0002/2011_09_26_drive_0002_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0005/2011_09_26_drive_0005_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0009/2011_09_26_drive_0009_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0011/2011_09_26_drive_0011_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0013/2011_09_26_drive_0013_sync.zip\n# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0014/2011_09_26_drive_0014_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0015/2011_09_26_drive_0015_sync.zip\n! wget https://s3.eu-central-1.amazonaws.com/avg-ki

In [6]:
import torch
! uname -a
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

Linux 8f8e2609b2c4 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux
True
Tesla T4


In [7]:
# ! python monodepth2/train.py --model_name stereo_model  --frame_ids 0 --use_stereo --split eigen_full

In [8]:
# ! find kitti_data/** -name '*.png' | parallel 'convert -quality 92 -sampling-factor 2x2,1x1,1x1 {.}.png {.}.jpg && rm {}'

In [9]:
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_calib.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0001/2011_09_28_drive_0001_sync.zip
# ! wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_28_drive_0002/2011_09_28_drive_0002_sync.zip



In [10]:
# ! rm -rf kitti_data
# ! mkdir kitti_data
# ! unzip -q 2011_09_28_drive_0001_sync.zip -d kitti_data
# ! rm -rf 2011_09_28_drive_0001_sync.zip
# ! mv data_temp/2011_09_26/2011_09_26_drive_0095_sync/* kitti_data

In [11]:
# ! sudo apt update
# ! sudo apt install imagemagick --fix-missing
# ! convert -h
# ! find kitti_data/ -name '*.png'
# ! sudo apt install parallel
# convert -quality 92 -sampling-factor 2x2,1x1,1x1 kitti_data/2011_09_26/2011_09_26_drive_0048_sync/image_02/data/0000000005.png jpg && rm {}
# ! find kitti_data/2011_09_28 -name '*.png' | parallel 'convert -quality 92 -sampling-factor 2x2,1x1,1x1 {.}.png {.}.jpg && rm {}'

In [ ]:
# ! python monodepth2/train.py --model_name S_640x192 --frame_ids 0 --use_stereo --pose_model_type separate_resnet --split eigen_full --data_path /content/kitti_data --num_epochs 10
# /content/kitti_data/2011_09_26/2011_09_26_drive_0106_sync/image_02/data/0000000115.png

False


In [ ]:
# ! zip -q -r /content/model.zip /root/tmp/S_640x192/*
# ! ls -la /root/tmp/S_640x192/models/

In [ ]:
# ! python monodepth2/export_gt_depth.py --data_path kitti_data --split eigen

In [ ]:
# ! python monodepth2/evaluate_depth.py --data_path kitti_data --load_weights_folder /root/tmp/S_640x192/models/weights_19/ --eval_stereo

In [ ]:

# #@title NETWORKS
# import torch.nn as nn
# import torchvision.models as models
# from collections import OrderedDict
# import torch.utils.model_zoo as model_zoo
# import skimage.transform

# class ResNetMultiImageInput(models.ResNet):
#     """Constructs a resnet model with varying number of input images.
#     Adapted from https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
#     """
#     def __init__(self, block, layers, num_classes=1000, num_input_images=1):
#         super(ResNetMultiImageInput, self).__init__(block, layers)
#         self.inplanes = 64
#         self.conv1 = nn.Conv2d(
#             num_input_images * 3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.layer1 = self._make_layer(block, 64, layers[0])
#         self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
#         self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)


# def resnet_multiimage_input(num_layers, pretrained=False, num_input_images=1):
#     """Constructs a ResNet model.
#     Args:
#         num_layers (int): Number of resnet layers. Must be 18 or 50
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         num_input_images (int): Number of frames stacked as input
#     """
#     assert num_layers in [18, 50], "Can only run with 18 or 50 layer resnet"
#     blocks = {18: [2, 2, 2, 2], 50: [3, 4, 6, 3]}[num_layers]
#     block_type = {18: models.resnet.BasicBlock, 50: models.resnet.Bottleneck}[num_layers]
#     model = ResNetMultiImageInput(block_type, blocks, num_input_images=num_input_images)

#     if pretrained:
#         loaded = model_zoo.load_url(models.resnet.model_urls['resnet{}'.format(num_layers)])
#         loaded['conv1.weight'] = torch.cat(
#             [loaded['conv1.weight']] * num_input_images, 1) / num_input_images
#         model.load_state_dict(loaded)
#     return model

# class Conv3x3(nn.Module):
#     """Layer to pad and convolve input
#     """
#     def __init__(self, in_channels, out_channels, use_refl=True):
#         super(Conv3x3, self).__init__()

#         if use_refl:
#             self.pad = nn.ReflectionPad2d(1)
#         else:
#             self.pad = nn.ZeroPad2d(1)
#         self.conv = nn.Conv2d(int(in_channels), int(out_channels), 3)

#     def forward(self, x):
#         out = self.pad(x)
#         out = self.conv(out)
#         return out


# class ConvBlock(nn.Module):
#     """Layer to perform a convolution followed by ELU
#     """
#     def __init__(self, in_channels, out_channels):
#         super(ConvBlock, self).__init__()

#         self.conv = Conv3x3(in_channels, out_channels)
#         self.nonlin = nn.ELU(inplace=True)

#     def forward(self, x):
#         out = self.conv(x)
#         out = self.nonlin(out)
#         return out

# class ResnetEncoder(nn.Module):
#     """Pytorch module for a resnet encoder
#     """
#     def __init__(self, num_layers, pretrained, num_input_images=1):
#         super(ResnetEncoder, self).__init__()

#         self.num_ch_enc = np.array([64, 64, 128, 256, 512])

#         resnets = {18: models.resnet18,
#                    34: models.resnet34,
#                    50: models.resnet50,
#                    101: models.resnet101,
#                    152: models.resnet152}

#         if num_layers not in resnets:
#             raise ValueError("{} is not a valid number of resnet layers".format(num_layers))

#         if num_input_images > 1:
#             self.encoder = resnet_multiimage_input(num_layers, pretrained, num_input_images)
#         else:
#             self.encoder = resnets[num_layers](pretrained)

#         if num_layers > 34:
#             self.num_ch_enc[1:] *= 4

#     def forward(self, input_image):
#         self.features = []
#         x = (input_image - 0.45) / 0.225
#         x = self.encoder.conv1(x)
#         x = self.encoder.bn1(x)
#         self.features.append(self.encoder.relu(x))
#         self.features.append(self.encoder.layer1(self.encoder.maxpool(self.features[-1])))
#         self.features.append(self.encoder.layer2(self.features[-1]))
#         self.features.append(self.encoder.layer3(self.features[-1]))
#         self.features.append(self.encoder.layer4(self.features[-1]))

#         return self.features

# class DepthDecoder(nn.Module):
#     def __init__(self, num_ch_enc, scales=range(4), num_output_channels=1, use_skips=True):
#         super(DepthDecoder, self).__init__()

#         self.num_output_channels = num_output_channels
#         self.use_skips = use_skips
#         self.upsample_mode = 'nearest'
#         self.scales = scales

#         self.num_ch_enc = num_ch_enc
#         self.num_ch_dec = np.array([16, 32, 64, 128, 256])

#         # decoder
#         self.convs = OrderedDict()
#         for i in range(4, -1, -1):
#             # upconv_0
#             num_ch_in = self.num_ch_enc[-1] if i == 4 else self.num_ch_dec[i + 1]
#             num_ch_out = self.num_ch_dec[i]
#             self.convs[("upconv", i, 0)] = ConvBlock(num_ch_in, num_ch_out)

#             # upconv_1
#             num_ch_in = self.num_ch_dec[i]
#             if self.use_skips and i > 0:
#                 num_ch_in += self.num_ch_enc[i - 1]
#             num_ch_out = self.num_ch_dec[i]
#             self.convs[("upconv", i, 1)] = ConvBlock(num_ch_in, num_ch_out)

#         for s in self.scales:
#             self.convs[("dispconv", s)] = Conv3x3(self.num_ch_dec[s], self.num_output_channels)

#         self.decoder = nn.ModuleList(list(self.convs.values()))
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, input_features):
#         self.outputs = {}

#         # decoder
#         x = input_features[-1]
#         for i in range(4, -1, -1):
#             x = self.convs[("upconv", i, 0)](x)
#             x = [upsample(x)]
#             if self.use_skips and i > 0:
#                 x += [input_features[i - 1]]
#             x = torch.cat(x, 1)
#             x = self.convs[("upconv", i, 1)](x)
#             if i in self.scales:
#                 self.outputs[("disp", i)] = self.sigmoid(self.convs[("dispconv", i)](x))

#         return self.outputs

# class PoseDecoder(nn.Module):
#     def __init__(self, num_ch_enc, num_input_features, num_frames_to_predict_for=None, stride=1):
#         super(PoseDecoder, self).__init__()

#         self.num_ch_enc = num_ch_enc
#         self.num_input_features = num_input_features

#         if num_frames_to_predict_for is None:
#             num_frames_to_predict_for = num_input_features - 1
#         self.num_frames_to_predict_for = num_frames_to_predict_for

#         self.convs = OrderedDict()
#         self.convs[("squeeze")] = nn.Conv2d(self.num_ch_enc[-1], 256, 1)
#         self.convs[("pose", 0)] = nn.Conv2d(num_input_features * 256, 256, 3, stride, 1)
#         self.convs[("pose", 1)] = nn.Conv2d(256, 256, 3, stride, 1)
#         self.convs[("pose", 2)] = nn.Conv2d(256, 6 * num_frames_to_predict_for, 1)

#         self.relu = nn.ReLU()

#         self.net = nn.ModuleList(list(self.convs.values()))

#     def forward(self, input_features):
#         last_features = [f[-1] for f in input_features]

#         cat_features = [self.relu(self.convs["squeeze"](f)) for f in last_features]
#         cat_features = torch.cat(cat_features, 1)

#         out = cat_features
#         for i in range(3):
#             out = self.convs[("pose", i)](out)
#             if i != 2:
#                 out = self.relu(out)

#         out = out.mean(3).mean(2)

#         out = 0.01 * out.view(-1, self.num_frames_to_predict_for, 1, 6)

#         axisangle = out[..., :3]
#         translation = out[..., 3:]

#         return axisangle, translation


In [ ]:
# import os
# import random
# import numpy as np
# import copy
# from PIL import Image  # using pillow-simd for increased speed

# import torch
# import torch.utils.data as data
# from torchvision import transforms
# from collections import Counter

# class BackprojectDepth(nn.Module):
#     """Layer to transform a depth image into a point cloud
#     """
#     def __init__(self, batch_size, height, width):
#         super(BackprojectDepth, self).__init__()

#         self.batch_size = batch_size
#         self.height = height
#         self.width = width

#         meshgrid = np.meshgrid(range(self.width), range(self.height), indexing='xy')
#         self.id_coords = np.stack(meshgrid, axis=0).astype(np.float32)
#         self.id_coords = nn.Parameter(torch.from_numpy(self.id_coords),
#                                       requires_grad=False)

#         self.ones = nn.Parameter(torch.ones(self.batch_size, 1, self.height * self.width),
#                                  requires_grad=False)

#         self.pix_coords = torch.unsqueeze(torch.stack(
#             [self.id_coords[0].view(-1), self.id_coords[1].view(-1)], 0), 0)
#         self.pix_coords = self.pix_coords.repeat(batch_size, 1, 1)
#         self.pix_coords = nn.Parameter(torch.cat([self.pix_coords, self.ones], 1),
#                                        requires_grad=False)

#     def forward(self, depth, inv_K):
#         cam_points = torch.matmul(inv_K[:, :3, :3], self.pix_coords)
#         cam_points = depth.view(self.batch_size, 1, -1) * cam_points
#         cam_points = torch.cat([cam_points, self.ones], 1)

#         return cam_points


# class Project3D(nn.Module):
#     """Layer which projects 3D points into a camera with intrinsics K and at position T
#     """
#     def __init__(self, batch_size, height, width, eps=1e-7):
#         super(Project3D, self).__init__()

#         self.batch_size = batch_size
#         self.height = height
#         self.width = width
#         self.eps = eps

#     def forward(self, points, K, T):
#         P = torch.matmul(K, T)[:, :3, :]

#         cam_points = torch.matmul(P, points)

#         pix_coords = cam_points[:, :2, :] / (cam_points[:, 2, :].unsqueeze(1) + self.eps)
#         pix_coords = pix_coords.view(self.batch_size, 2, self.height, self.width)
#         pix_coords = pix_coords.permute(0, 2, 3, 1)
#         pix_coords[..., 0] /= self.width - 1
#         pix_coords[..., 1] /= self.height - 1
#         pix_coords = (pix_coords - 0.5) * 2
#         return pix_coords

# def pil_loader(path):
#     # open path as file to avoid ResourceWarning
#     # (https://github.com/python-pillow/Pillow/issues/835)
#     with open(path, 'rb') as f:
#         with Image.open(f) as img:
#             return img.convert('RGB')


# class MonoDataset(data.Dataset):
#     """Superclass for monocular dataloaders

#     Args:
#         data_path
#         filenames
#         height
#         width
#         frame_idxs
#         num_scales
#         is_train
#         img_ext
#     """
#     def __init__(self,
#                  data_path,
#                  filenames,
#                  height,
#                  width,
#                  frame_idxs,
#                  num_scales,
#                  is_train=False,
#                  img_ext='.jpg'):
#         super(MonoDataset, self).__init__()

#         self.data_path = data_path
#         self.filenames = filenames
#         self.height = height
#         self.width = width
#         self.num_scales = num_scales
#         self.interp = Image.ANTIALIAS

#         self.frame_idxs = frame_idxs

#         self.is_train = is_train
#         self.img_ext = img_ext

#         self.loader = pil_loader
#         self.to_tensor = transforms.ToTensor()

#         # We need to specify augmentations differently in newer versions of torchvision.
#         # We first try the newer tuple version; if this fails we fall back to scalars
#         try:
#             self.brightness = (0.8, 1.2)
#             self.contrast = (0.8, 1.2)
#             self.saturation = (0.8, 1.2)
#             self.hue = (-0.1, 0.1)
#             transforms.ColorJitter.get_params(
#                 self.brightness, self.contrast, self.saturation, self.hue)
#         except TypeError:
#             self.brightness = 0.2
#             self.contrast = 0.2
#             self.saturation = 0.2
#             self.hue = 0.1

#         self.resize = {}
#         for i in range(self.num_scales):
#             s = 2 ** i
#             self.resize[i] = transforms.Resize((self.height // s, self.width // s),
#                                                interpolation=self.interp)

#         self.load_depth = self.check_depth()

#     def preprocess(self, inputs, color_aug):
#         """Resize colour images to the required scales and augment if required

#         We create the color_aug object in advance and apply the same augmentation to all
#         images in this item. This ensures that all images input to the pose network receive the
#         same augmentation.
#         """
#         for k in list(inputs):
#             frame = inputs[k]
#             if "color" in k:
#                 n, im, i = k
#                 for i in range(self.num_scales):
#                     inputs[(n, im, i)] = self.resize[i](inputs[(n, im, i - 1)])

#         for k in list(inputs):
#             f = inputs[k]
#             if "color" in k:
#                 n, im, i = k
#                 inputs[(n, im, i)] = self.to_tensor(f)
#                 inputs[(n + "_aug", im, i)] = self.to_tensor(color_aug(f))

#     def __len__(self):
#         return len(self.filenames)

#     def __getitem__(self, index):
#         """Returns a single training item from the dataset as a dictionary.

#         Values correspond to torch tensors.
#         Keys in the dictionary are either strings or tuples:

#             ("color", <frame_id>, <scale>)          for raw colour images,
#             ("color_aug", <frame_id>, <scale>)      for augmented colour images,
#             ("K", scale) or ("inv_K", scale)        for camera intrinsics,
#             "stereo_T"                              for camera extrinsics, and
#             "depth_gt"                              for ground truth depth maps.

#         <frame_id> is either:
#             an integer (e.g. 0, -1, or 1) representing the temporal step relative to 'index',
#         or
#             "s" for the opposite image in the stereo pair.

#         <scale> is an integer representing the scale of the image relative to the fullsize image:
#             -1      images at native resolution as loaded from disk
#             0       images resized to (self.width,      self.height     )
#             1       images resized to (self.width // 2, self.height // 2)
#             2       images resized to (self.width // 4, self.height // 4)
#             3       images resized to (self.width // 8, self.height // 8)
#         """
#         inputs = {}

#         do_color_aug = self.is_train and random.random() > 0.5
#         do_flip = self.is_train and random.random() > 0.5

#         line = self.filenames[index].split()
#         folder = line[0]

#         if len(line) == 3:
#             frame_index = int(line[1])
#         else:
#             frame_index = 0

#         if len(line) == 3:
#             side = line[2]
#         else:
#             side = None

#         for i in self.frame_idxs:
#             if i == "s":
#                 other_side = {"r": "l", "l": "r"}[side]
#                 inputs[("color", i, -1)] = self.get_color(folder, frame_index, other_side, do_flip)
#             else:
#                 inputs[("color", i, -1)] = self.get_color(folder, frame_index + i, side, do_flip)

#         # adjusting intrinsics to match each scale in the pyramid
#         for scale in range(self.num_scales):
#             K = self.K.copy()

#             K[0, :] *= self.width // (2 ** scale)
#             K[1, :] *= self.height // (2 ** scale)

#             inv_K = np.linalg.pinv(K)

#             inputs[("K", scale)] = torch.from_numpy(K)
#             inputs[("inv_K", scale)] = torch.from_numpy(inv_K)

#         if do_color_aug:
#             color_aug = transforms.ColorJitter.get_params(
#                 self.brightness, self.contrast, self.saturation, self.hue)
#         else:
#             color_aug = (lambda x: x)

#         self.preprocess(inputs, color_aug)

#         for i in self.frame_idxs:
#             del inputs[("color", i, -1)]
#             del inputs[("color_aug", i, -1)]

#         if self.load_depth:
#             depth_gt = self.get_depth(folder, frame_index, side, do_flip)
#             inputs["depth_gt"] = np.expand_dims(depth_gt, 0)
#             inputs["depth_gt"] = torch.from_numpy(inputs["depth_gt"].astype(np.float32))

#         if "s" in self.frame_idxs:
#             stereo_T = np.eye(4, dtype=np.float32)
#             baseline_sign = -1 if do_flip else 1
#             side_sign = -1 if side == "l" else 1
#             stereo_T[0, 3] = side_sign * baseline_sign * 0.1

#             inputs["stereo_T"] = torch.from_numpy(stereo_T)

#         return inputs

#     def get_color(self, folder, frame_index, side, do_flip):
#         raise NotImplementedError

#     def check_depth(self):
#         raise NotImplementedError

#     def get_depth(self, folder, frame_index, side, do_flip):
#         raise NotImplementedError

# class KITTIDataset(MonoDataset):
#     """Superclass for different types of KITTI dataset loaders
#     """
#     def __init__(self, *args, **kwargs):
#         super(KITTIDataset, self).__init__(*args, **kwargs)

#         # NOTE: Make sure your intrinsics matrix is *normalized* by the original image size    
#         self.K = np.array([[0.58, 0, 0.5, 0],
#                            [0, 1.92, 0.5, 0],
#                            [0, 0, 1, 0],
#                            [0, 0, 0, 1]], dtype=np.float32)

#         self.full_res_shape = (1242, 375)
#         self.side_map = {"2": 2, "3": 3, "l": 2, "r": 3}

#     def check_depth(self):
#         line = self.filenames[0].split()
#         scene_name = line[0]
#         frame_index = int(line[1])

#         velo_filename = os.path.join(
#             self.data_path,
#             scene_name,
#             "velodyne_points/data/{:010d}.bin".format(int(frame_index)))

#         return os.path.isfile(velo_filename)

#     def get_color(self, folder, frame_index, side, do_flip):
#         color = self.loader(self.get_image_path(folder, frame_index, side))

#         if do_flip:
#             color = color.transpose(pil.FLIP_LEFT_RIGHT)

#         return color


# class KITTIRAWDataset(KITTIDataset):
#     """KITTI dataset which loads the original velodyne depth maps for ground truth
#     """
#     def __init__(self, *args, **kwargs):
#         super(KITTIRAWDataset, self).__init__(*args, **kwargs)

#     def get_image_path(self, folder, frame_index, side):
#         f_str = "{:010d}{}".format(frame_index, self.img_ext)
#         image_path = os.path.join(
#             self.data_path, folder, "image_0{}/data".format(self.side_map[side]), f_str)
#         return image_path

#     def get_depth(self, folder, frame_index, side, do_flip):
#         calib_path = os.path.join(self.data_path, folder.split("/")[0])

#         velo_filename = os.path.join(
#             self.data_path,
#             folder,
#             "velodyne_points/data/{:010d}.bin".format(int(frame_index)))

#         depth_gt = generate_depth_map(calib_path, velo_filename, self.side_map[side])
#         depth_gt = skimage.transform.resize(
#             depth_gt, self.full_res_shape[::-1], order=0, preserve_range=True, mode='constant')

#         if do_flip:
#             depth_gt = np.fliplr(depth_gt)

#         return depth_gt

# class SSIM(nn.Module):
#     """Layer to compute the SSIM loss between a pair of images
#     """
#     def __init__(self):
#         super(SSIM, self).__init__()
#         self.mu_x_pool   = nn.AvgPool2d(3, 1)
#         self.mu_y_pool   = nn.AvgPool2d(3, 1)
#         self.sig_x_pool  = nn.AvgPool2d(3, 1)
#         self.sig_y_pool  = nn.AvgPool2d(3, 1)
#         self.sig_xy_pool = nn.AvgPool2d(3, 1)

#         self.refl = nn.ReflectionPad2d(1)

#         self.C1 = 0.01 ** 2
#         self.C2 = 0.03 ** 2

#     def forward(self, x, y):
#         x = self.refl(x)
#         y = self.refl(y)

#         mu_x = self.mu_x_pool(x)
#         mu_y = self.mu_y_pool(y)

#         sigma_x  = self.sig_x_pool(x ** 2) - mu_x ** 2
#         sigma_y  = self.sig_y_pool(y ** 2) - mu_y ** 2
#         sigma_xy = self.sig_xy_pool(x * y) - mu_x * mu_y

#         SSIM_n = (2 * mu_x * mu_y + self.C1) * (2 * sigma_xy + self.C2)
#         SSIM_d = (mu_x ** 2 + mu_y ** 2 + self.C1) * (sigma_x + sigma_y + self.C2)

#         return torch.clamp((1 - SSIM_n / SSIM_d) / 2, 0, 1)

# def generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False):
#     """Generate a depth map from velodyne data
#     """
#     # load calibration files
#     cam2cam = read_calib_file(os.path.join(calib_dir, 'calib_cam_to_cam.txt'))
#     velo2cam = read_calib_file(os.path.join(calib_dir, 'calib_velo_to_cam.txt'))
#     velo2cam = np.hstack((velo2cam['R'].reshape(3, 3), velo2cam['T'][..., np.newaxis]))
#     velo2cam = np.vstack((velo2cam, np.array([0, 0, 0, 1.0])))

#     # get image shape
#     im_shape = cam2cam["S_rect_02"][::-1].astype(np.int32)

#     # compute projection matrix velodyne->image plane
#     R_cam2rect = np.eye(4)
#     R_cam2rect[:3, :3] = cam2cam['R_rect_00'].reshape(3, 3)
#     P_rect = cam2cam['P_rect_0'+str(cam)].reshape(3, 4)
#     P_velo2im = np.dot(np.dot(P_rect, R_cam2rect), velo2cam)

#     # load velodyne points and remove all behind image plane (approximation)
#     # each row of the velodyne data is forward, left, up, reflectance
#     velo = load_velodyne_points(velo_filename)
#     velo = velo[velo[:, 0] >= 0, :]

#     # project the points to the camera
#     velo_pts_im = np.dot(P_velo2im, velo.T).T
#     velo_pts_im[:, :2] = velo_pts_im[:, :2] / velo_pts_im[:, 2][..., np.newaxis]

#     if vel_depth:
#         velo_pts_im[:, 2] = velo[:, 0]

#     # check if in bounds
#     # use minus 1 to get the exact same value as KITTI matlab code
#     velo_pts_im[:, 0] = np.round(velo_pts_im[:, 0]) - 1
#     velo_pts_im[:, 1] = np.round(velo_pts_im[:, 1]) - 1
#     val_inds = (velo_pts_im[:, 0] >= 0) & (velo_pts_im[:, 1] >= 0)
#     val_inds = val_inds & (velo_pts_im[:, 0] < im_shape[1]) & (velo_pts_im[:, 1] < im_shape[0])
#     velo_pts_im = velo_pts_im[val_inds, :]

#     # project to image
#     depth = np.zeros((im_shape[:2]))
#     depth[velo_pts_im[:, 1].astype(np.int), velo_pts_im[:, 0].astype(np.int)] = velo_pts_im[:, 2]

#     # find the duplicate points and choose the closest depth
#     inds = sub2ind(depth.shape, velo_pts_im[:, 1], velo_pts_im[:, 0])
#     dupe_inds = [item for item, count in Counter(inds).items() if count > 1]
#     for dd in dupe_inds:
#         pts = np.where(inds == dd)[0]
#         x_loc = int(velo_pts_im[pts[0], 0])
#         y_loc = int(velo_pts_im[pts[0], 1])
#         depth[y_loc, x_loc] = velo_pts_im[pts, 2].min()
#     depth[depth < 0] = 0

#     return depth

# def read_calib_file(path):
#     """Read KITTI calibration file
#     (from https://github.com/hunse/kitti)
#     """
#     float_chars = set("0123456789.e+- ")
#     data = {}
#     with open(path, 'r') as f:
#         for line in f.readlines():
#             key, value = line.split(':', 1)
#             value = value.strip()
#             data[key] = value
#             if float_chars.issuperset(value):
#                 # try to cast to float array
#                 try:
#                     data[key] = np.array(list(map(float, value.split(' '))))
#                 except ValueError:
#                     # casting error: data[key] already eq. value, so pass
#                     pass

#     return data

# def load_velodyne_points(filename):
#     """Load 3D point cloud from KITTI file format
#     (adapted from https://github.com/hunse/kitti)
#     """
#     points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
#     points[:, 3] = 1.0  # homogeneous
#     return points

# def sub2ind(matrixSize, rowSub, colSub):
#     """Convert row, col matrix subscripts to linear indices
#     """
#     m, n = matrixSize
#     return rowSub * (n-1) + colSub - 1


In [35]:
#@title Default title text
### For Training
import monodepth2.layers, monodepth2.utils, monodepth2.trainer
import monodepth2.networks

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import timeit
import PIL.Image as pil


class MyTraining(object):
    def __init__(self):
        self.h = 192
        self.w = 640
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.frame_diffs = [0, 's']

        self.loss_scales = [0, 1, 2, 3]

        # Define projections for pose estimation
        self.depth_projection = {}
        self.projection_3d = {}
        for loss_scale in self.loss_scales:
            h = int(self.h / (2 ** loss_scale))
            w = int(self.w / (2 ** loss_scale))

            self.depth_projection[loss_scale] = monodepth2.layers.BackprojectDepth(6, h, w).to(self.device)
            self.projection_3d[loss_scale] = monodepth2.layers.Project3D(6, h, w).to(self.device)
        
        
        # Declare 2 Neural Networks

        ## Declare Depth Network
        self.depth_encoder_network = monodepth2.networks.ResnetEncoder(18, True).to(self.device)
        self.depth_decoder_network = monodepth2.networks.DepthDecoder(self.depth_encoder_network.num_ch_enc, 
                                                [0, 1, 2, 3]).to(self.device) 

        ## Declare Pose Network
        self.pose_encoder_network = monodepth2.networks.ResnetEncoder(18, True, num_input_images=2).to(self.device)
        self.pose_decoder_network = monodepth2.networks.PoseDecoder(self.pose_encoder_network.num_ch_enc, 
                                                num_input_features=1, num_frames_to_predict_for=2).to(self.device)

        self.models = {"encoder": self.depth_encoder_network, "depth": self.depth_decoder_network,
                       "pose_encoder": self.pose_encoder_network, "pose": self.pose_decoder_network}

        # Predictive Mask Not Used

        # Not using load_model feature

        # Set up our excerpt of Kitti Dataset
        training_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/train_files.txt")
        validation_files = monodepth2.utils.readlines("/content/monodepth2/splits/eigen_full/val_files.txt")

        train_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", training_files, self.h, self.w, self.frame_diffs, 4, is_train=True, img_ext='.jpg')
        self.train_loader = DataLoader(train_set, 6, True, num_workers=1, pin_memory=True, drop_last=True)
        val_set = monodepth2.datasets.kitti_dataset.KITTIRAWDataset("/content/kitti_data", validation_files, self.h, self.w, self.frame_diffs, 4, is_train=False, img_ext='.jpg')
        self.val_loader = DataLoader(val_set, 6, True, num_workers=1, pin_memory=True, drop_last=True)

    def run_training_loop(self):
        self.all_params = list(self.depth_encoder_network.parameters())
        self.all_params += list(self.depth_decoder_network.parameters())
        self.all_params += list(self.pose_encoder_network.parameters())
        self.all_params += list(self.pose_decoder_network.parameters())

        self.adam_optim = optim.Adam(self.all_params, 1e-4)
        self.lr_sched = optim.lr_scheduler.StepLR(self.adam_optim, 15, 0.1)
        self.ssim_loss_func = monodepth2.layers.SSIM().to(self.device)

        self.epoch_count = 0
        self.step_count = 0
        self.start = timeit.default_timer()

        losses  = []

        for self.epoch_count in range(2):
            # Per Epoch
            self.lr_sched.step()
            self.depth_encoder_network.train()
            self.depth_decoder_network.train()
            self.pose_encoder_network.train()
            self.pose_decoder_network.train()

            for idx, inputs in enumerate(self.train_loader):
                for key, ipt in inputs.items():
                    inputs[key] = ipt.to(self.device)

                # Per Batch Code
                batch_start_time = timeit.default_timer()

                feature_identifications = self.depth_encoder_network(inputs["color_aug", 0, 0])
                outputs = self.depth_decoder_network(feature_identifications)

                self.estimate_stereo_predictions(inputs, outputs)
                loss = self.batch_loss_func(inputs, outputs)

                self.adam_optim.zero_grad()
                loss["loss"].backward()
                losses.append(loss["loss"].item())
                self.adam_optim.step()

                batch_duration = timeit.default_timer() - batch_start_time

                # Do something with batch duration and losses

                self.step_count += 1

    def estimate_stereo_predictions(self, inputs, outputs):
        # Generate estimated stereo pair using the pose networks
        for loss_scale in self.loss_scales:
            estimated_disparity = outputs[("disp", loss_scale)]
            disp = F.interpolate(estimated_disparity, [self.h, self.w], mode="bilinear", align_corners=False)
            base = 0 # base scale

            # Change names
            scaled_disp = 0.001 + (10 - 0.001) * disp
            depth = 1 / scaled_disp

            outputs[("depth", 0, loss_scale)] = depth

            # Tweak this code once I understand things a bit more
            for idx, id in enumerate(self.frame_diffs[1:]):
                if id == "s":
                    T = inputs["stereo_T"]
                else:
                    T = outputs[("cam_T_cam", 0, id)]

                camera_coords = self.depth_projection[base](depth, inputs[("inv_K", base)])
                pixel_coords = self.projection_3d[base](camera_coords, inputs[("K", base)], T)

                outputs[("sample", id, loss_scale)] = pixel_coords
                outputs[("color", id, loss_scale)] = F.grid_sample(
                    inputs[("color", id, base)], outputs[("sample", id, loss_scale)], padding_mode="border")
                
    def reprojection(self, prediction, target):
        ssim_loss = self.ssim_loss_func(prediction, target).mean(1, True)
        reproj_losses = 0.85*ssim_loss + 0.15*(torch.abs(target-prediction).mean(1, True))
        return reproj_losses

    def batch_loss_func(self, inputs, outputs):
        batch_loss = {}
        complete_losses = 0

        for loss_scale in self.loss_scales:
            current_loss = 0

            base = 0

            reproj_losses = []

            for id in self.frame_diffs[1:]:
                reproj_losses.append(self.reprojection(outputs[("color", id, loss_scale)], inputs[("color", 0, base)]))
            reproj_losses = torch.cat(reproj_losses, 1)

            identity_loss = []
            for id in self.frame_diffs[1:]:
                identity_loss.append(self.reprojection(inputs[("color", id, base)], inputs[("color", 0, base)]))
            identity_loss = torch.cat(identity_loss, 1)


            # Add some minor noise to ensure no repeated values
            ## REDO THIS WITH NUMPY
            identity_loss += torch.rand(identity_loss.shape).cuda() * 0.00001
            total = torch.cat((identity_loss, reproj_losses), dim=1)
            # Check if this is always executed
            val, idxs = torch.min(total, dim=1)

            # UNDERSTAND THIS LINE MORE
            outputs["identity_selection/{}".format(loss_scale)] = (idxs > identity_loss.shape[1] - 1).float()

            current_loss += val.mean()

            mean_disp = outputs[("disp", loss_scale)].mean(2, True).mean(3, True)
            norm_disp = outputs[("disp", loss_scale)] / (mean_disp + 1e-7)
            smooth_loss = monodepth2.layers.get_smooth_loss(norm_disp, inputs[("color", 0, loss_scale)])

            current_loss += 1e-3 * smooth_loss / (2 ** loss_scale)
            complete_losses += current_loss
            batch_loss["loss/{}".format(loss_scale)] = current_loss

        complete_losses /= len(self.loss_scales)
        batch_loss["loss"] = complete_losses
        return batch_loss


In [37]:
import os
import torchvision.models as models
# def save_model(train):
#     """Save model weights to disk
#     """
#     save_folder = os.path.join("/content/test_model/")
#     if not os.path.exists(save_folder):
#         os.makedirs(save_folder)

#     save_path_de = os.path.join(save_folder, "depth_encoder.pth")
#     de = train.depth_encoder_network.state_dict()
#     de['height'] = train.h
#     de['width'] = train.w
#     de['use_stereo'] = True    
#     torch.save(de, save_path_de)
    
#     save_path_dd = os.path.join(save_folder, "depth_decoder.pth")
#     torch.save(train.depth_decoder_network.state_dict(), save_path_dd)
    
#     save_path_pe = os.path.join(save_folder, "pose_encoder.pth")
#     pe = train.pose_encoder_network.state_dict()
#     pe['height'] = train.h
#     pe['width'] = train.w
#     pe['use_stereo'] = True    
#     torch.save(pe, save_path_pe)

#     save_path_pd = os.path.join(save_folder, "pose_decoder.pth")
#     torch.save(train.pose_decoder_network.state_dict(), save_path_pd)
#     to_save = train.pose_encoder_network.state_dict()

#     save_path = os.path.join(save_folder, "{}.pth".format("adam"))
#     torch.save(train.adam_optim.state_dict(), save_path)

# save_model(train)

def save_model(train):
    """Save model weights to disk
    """
    save_folder = os.path.join("/content/test_model/")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for model_name, model in train.models.items():
        save_path = os.path.join(save_folder, "{}.pth".format(model_name))
        to_save = model.state_dict()
        if 'encoder' in model_name:
            # save the sizes - these are needed at prediction time
            to_save['height'] = train.h
            to_save['width'] = train.w
            to_save['use_stereo'] = True
        torch.save(to_save, save_path)

    save_path = os.path.join(save_folder, "{}.pth".format("adam"))
    torch.save(train.adam_optim.state_dict(), save_path)

In [ ]:
train = MyTraining()
train.run_training_loop()
save_model(train)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [25]:
model_path = os.path.join("/content/test_model/")
